In [8]:
import nltk
import pandas as pd
import math
import time

#Read data from existing CSV file and write into a text file 
"""-----------------------------------------------------------------"""
dataframe = pd.read_csv("data.csv", header =0, usecols=["Outcome", "Salesforce Notes"])
dataframe.to_csv("notesOutput.txt", 
                 columns=["Outcome", "Salesforce Notes"], 
                 header=["Autoclose", "Notes"], 
                 sep=' ',
                 index=None)


"""-----------------------------------------------------------------"""
#Apply TDF

start = time.time()
#.1 Extract documents (notes) from notesOutput.txt file
dataframe = pd.read_csv("notesOutput.txt", header=None, sep=' ', skiprows=[0])
documents = dataframe[dataframe.columns[1:2]]

#.2 Extract set of unique tokens from all the documents and prepare dict containing set of unique words and all words

set_of_unique_tokens = []
corpus_dict ={}

corpus = [doc[1] for doc in documents.itertuples()]

def decode(s, encoding="ascii", errors="ignore"):
    return s.decode(encoding=encoding, errors=errors)

def getAllWords(document):
    all_words =[]
    sentences = nltk.sent_tokenize(decode(str(document)))
    for sentence in sentences:
        word_token = nltk.word_tokenize(sentence)
        all_words.extend(word_token)
    #all_words = str(document[1]).split(' ')
    return all_words

for index ,document in enumerate(corpus):
    all_words = getAllWords(document)
    unique_words = set(all_words)
    doc_details ={'unique_words': unique_words, 'all_words': all_words}
    #print doc_details
    corpus_dict[index] = doc_details

x = [(val['unique_words'])for val in corpus_dict.values()]
set_of_unique_tokens = set.union(*x)

#print len(set_of_unique_tokens)
            
def calculateIDF(token):
    total_docs = len(documents)
    no_of_docs_having_token =0
    for val in corpus_dict.values():
        if token in val['unique_words']:
            no_of_docs_having_token+=1
    idf = math.log(total_docs / float(no_of_docs_having_token))
    #print('index: %d token: %s   no_of_docs: %s   idf:%s'% (index, token , no_of_docs_having_token, idf))
    return idf

def calculateTF(token , doc_words):
    word_freq = doc_words.count(token) 
    total_words = len(doc_words)
    tf = word_freq / float(total_words)
    return tf


results_dict = []
for token in set_of_unique_tokens:
    idf_token = calculateIDF(token)
    docIndex =0
    
    for val in corpus_dict.values():
        tf_token = calculateTF(token , val['all_words'])
        tf_idf =  tf_token * idf_token
        docIndex+=1
        results_dict.append({'docIndex': docIndex, 'token': token , 'tf_idf': tf_idf})
end = time.time()
print ("Total time taken:%s "% (end-start))


res_dataFrame = pd.DataFrame(results_dict)
text = res_dataFrame.to_string()

f = open('results.txt', 'w+')
f.write(text)
f.close()


print "Processing Finished"

Total time taken:8.62282299995 
Processing Finished
